In [1]:
import pandas as pd
import psycopg2

In [5]:
# Function to create database
def create_database():
    
    #Establish connection with default database
    conn = psycopg2.connect(
        host = '127.0.0.1',
        dbname = 'postgres',
        user = 'postgres',
        password = 'root'
    )
    
    conn.set_session( autocommit= True) # automate commit
    cur = conn.cursor()   # get cursor
    
    # Create database
    conn.execute("DROP DATABASE education")  # drop incase db existed
    conn.execute("CREATE DATABASE education")
    
    # Close connection to default database
    cur.close()
    
    # connect to the created database
    conn = psycopg2.connect(
        host = '127.0.0.1',
        dbname = 'education',
        user = 'postgres',
        password = 'root'
        )
    
    cur =conn.cursor()
    
    return cur, conn
    
    

In [ ]:
# function to delete table
def drop_table(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
# function to create table
def create_table(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [146]:
TeacherStatus =pd.read_csv('Data/teacher-by-status-and-county-for-secondary-csv-1.csv')
TeacherCounty = pd.read_csv ('Data/teachers-per-county-in-kenya-csv-2.csv')

In [147]:
TeacherStatus.head()

,COUNTY,School Type,Employment Body,No. of Teachers,County (centroid),Year
0,Baringo,Public,TSC,1172,"(0.669252, 35.946465)",12/31/2014 12:00:00 AM
1,Bomet,Public,TSC,1379,"(-0.726295, 35.298598)",12/31/2014 12:00:00 AM
2,Bungoma,Public,TSC,2946,"(0.749285, 34.640461)",12/31/2014 12:00:00 AM
3,Busia,Public,TSC,1182,"(0.387444, 34.193631)",12/31/2014 12:00:00 AM
4,Elgeyo Marakwet,Public,TSC,1087,"(0.802219, 35.536563)",12/31/2014 12:00:00 AM


In [152]:
TeacherStatus['Year'] = pd.to_datetime(TeacherStatus['Year'])
TeacherCounty['Year'] = pd.to_datetime(TeacherCounty['Year'])

In [153]:
TeacherStatus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   COUNTY             192 non-null    object        
 1   School Type        192 non-null    object        
 2   Employment Body    192 non-null    object        
 3   No. of Teachers    192 non-null    int64         
 4   County (centroid)  188 non-null    object        
 5   Year               192 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 9.1+ KB


In [154]:
TeacherCounty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   County             194 non-null    object        
 1   Public             194 non-null    object        
 2   Total Number       192 non-null    float64       
 3   County (Centroid)  188 non-null    object        
 4   Year               194 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 7.7+ KB


In [157]:
print("Shape of the TeacherStatus: ",TeacherStatus.shape)
print("Shape of the TeacherCounty: ", TeacherCounty.shape)

Shape of the TeacherStatus:  (192, 6)
Shape of the TeacherCounty:  (194, 5)


In [158]:
TeacherCounty.head()

,County,Public,Total Number,County (Centroid),Year
0,Baringo,TSC,5188.0,"(0.669252, 35.9464654)",2014-12-31
1,Bomet,TSC,4787.0,"(-0.7262946, 35.2985976)",2014-12-31
2,Bungoma,TSC,8327.0,"(0.7492852, 34.6404609)",2014-12-31
3,Busia,TSC,4659.0,"(0.3874437, 34.1936315)",2014-12-31
4,Elgeyo Marakwet,TSC,3511.0,"(0.8022188, 35.5365631)",2014-12-31


In [159]:
# Rename columns
TeacherStatus.rename({
     "COUNTY":"County",
     "No. of Teachers": "No of Teachers"    
 
},axis = 1 , inplace = True 
)

In [160]:
# rename columns
TeacherCounty.rename({ "Public": "Employment Body"}, axis =1 , inplace = True)

In [161]:
TeacherCounty.nunique()

County                49
Employment Body        5
Total Number         189
County (Centroid)     47
Year                   1
dtype: int64

In [162]:
TeacherStatus.nunique()

County                48
School Type            2
Employment Body        3
No of Teachers       141
County (centroid)     47
Year                   1
dtype: int64

### Extract Unique Dimension Data

In [50]:
# Extract unique values for dimensions
school_types = TeacherStatus['School Type'].unique()
employment_bodies = TeacherStatus['Employment Body'].unique()

In [54]:
# Prepare unique dataframes for dimension tables

school_type_df = pd.DataFrame(school_types, columns=['School Type'])
employment_body_df = pd.DataFrame(employment_bodies, columns=['Employment Body'])

In [80]:
# Extract years
years = TeacherStatus['Year'].unique()
year_df = pd.DataFrame(years, columns=['Year'])

In [77]:
# Fetch data for the county and their codes from url
import requests
from bs4 import BeautifulSoup 


url = "https://en.wikipedia.org/wiki/Counties_of_Kenya" # URL containing the list of Kenyan counties

# Fetch the list of counties from the URL
response = requests.get(url)
response.raise_for_status()  # Raise an error for bad responses

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with counties (assumed to be the first table on the page)
table = soup.find('table', {'class': 'wikitable'})

In [78]:
# Extract counties from the table
data = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) >= 3:  # Ensure there are enough columns
        county_data = {
            'Code': cells[0].get_text(strip=True),
            'County': cells[1].get_text(strip=True)
            
        }
        data.append(county_data)

# Convert to DataFrame
counties_data = pd.DataFrame(data)
counties_data.set_index('Code', inplace=True)

In [105]:
counties_data

,County
Code,
1,Mombasa
2,Kwale
3,Kilifi
4,Tana River
5,Lamu
6,Taita–Taveta
7,Garissa
8,Wajir
9,Mandera


In [125]:
counties_data["County"] = counties_data["County"].replace(
    {
        "Elgeyo-Marakwet" : 'Elgeyo Marakwet',
        "Trans-Nzoia": 'Trans Nzoia',
        "Murang'A" : 'Muranga',
        "Taita–Taveta": 'Taita Taveta'
      
        
    }
)

In [126]:
counties_data

,County
Code,
1,Mombasa
2,Kwale
3,Kilifi
4,Tana River
5,Lamu
6,Taita Taveta
7,Garissa
8,Wajir
9,Mandera


In [127]:
#Extract unique counties from the TeacherStatus dataframe
counties = TeacherStatus['County'].unique()
county_df = pd.DataFrame(counties, columns=['County']) 

In [142]:
# Drop rows where 'County' column contains the value 'Kenya'
county_df  = county_df [county_df ['County'] != 'Kenya']

In [143]:
# Normalize the county names if necessary
counties_data['County'] = counties_data['County'].str.strip().str.title()
county_df ['County'] = county_df ['County'].str.strip().str.title()

# Check if all counties in county_df are present in counties_data
missing_counties = set((county_df['County'] - counties_data['County']))
if missing_counties:
    print(f"Warning: The following counties are missing from the counties_data DataFrame: {missing_counties}")
else:
    # Create a mapping of county names to codes
    name_to_code = counties_data['County'].reset_index().set_index('County').to_dict()['Code']
    
    # Map the counties in county_df to their codes
    county_df['Code'] = county_df['County'].map(name_to_code)
    
    # Display the resulting DataFrame
    print(county_df)


C:\Users\gitom\AppData\Local\Temp\ipykernel_3952\3811022754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df ['County'] = county_df ['County'].str.strip().str.title()


In [141]:
counties_data

,County
Code,
1,Mombasa
2,Kwale
3,Kilifi
4,Tana River
5,Lamu
6,Taita Taveta
7,Garissa
8,Wajir
9,Mandera


,County
Code,
1,Mombasa
2,Kwale
3,Kilifi
4,Tana River
5,Lamu
6,Taita–Taveta
7,Garissa
8,Wajir
9,Mandera


In [112]:
county_df

,County
0,Baringo
1,Bomet
2,Bungoma
3,Busia
4,Elgeyo Marakwet
5,Embu
6,Garissa
7,Homa Bay
8,Isiolo
9,Kajiado


In [85]:


# Create a mapping of counties to their codes
county_to_code = counties_data['Code'].to_dict()

# Map the counties to their codes in the teacher_status_df
county_df['Code'] = county_df['County'].map(county_to_code)

KeyError: 'Code'

In [ ]:
# Comparing the data fetched from the TeacherStatus dataframe if it matches with Actual County Names and Codes



In [32]:
# Dimension tables

dimension_tables ="""
CREATE TABLE IF NOT EXISTS County (
    County_ID SERIAL PRIMARY KEY,
    County VARCHAR(255),
    County_Centroid GEOGRAPHY
);

CREATE TABLE IF NOT EXISTS School_Type (
    School_Type_ID SERIAL PRIMARY KEY,
    School_Type VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS Employment_Body (
    Employment_Body_ID SERIAL PRIMARY KEY,
    Employment_Body VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS Year (
    Year_ID SERIAL PRIMARY KEY,
    Year DATETIME
);

"""

In [ ]:
cur.execute(dimension_tables)
conn.commit()

In [33]:
# Fact table

fact_table = """

CREATE TABLE IF NOT EXISTS Fact_Table (
    Fact_ID SERIAL PRIMARY KEY,
    County_ID INT REFERENCES County(County_ID),
    School_Type_ID INT REFERENCES School_Type(School_Type_ID),
    Employment_Body_ID INT REFERENCES Employment_Body(Employment_Body_ID),
    Year_ID INT REFERENCES Year(Year_ID),
    No_of_Teachers INT,
    Total_Number INT
);


"""

In [ ]:
cur.execute(fact_table)
conn.commit()